# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

## Python ETL Sandpit Codes

In [6]:
import os
import glob
import json
import psycopg2
import pandas as pd
import numpy as np
from typing import List
from sql_queries import *

In [7]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [9]:
type(conn)

psycopg2.extensions.connection

In [3]:
# provided in original udacity template.
def get_files(filepath):
    """given a root filepath, retrieve all JSON file paths nested wtihin the root filepath
    and store into a list """
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [4]:
# (NO LONGER USED) created this function during experiment. Required to ensure smooth parsing from dataframe to psql
def convert_np_fields_to_py_native(panda_like_record):
    """Check a pandas record and convert fields into native types
    Motivation: if a field is of numpy.integer or numpy.float type, we would get an error
    can't adapt type XXXX (such as 'numpy.int32', 'numpy.int64', 'numpy.float32', 'numpy.float64')
    This step ensure the record contains only native Python types so psycopg2 / postgres would understand.
    """
    new_list = []
    for idx, field in enumerate(panda_like_record):
        if np.issubdtype(type(field), np.integer):
            new_list.append(int(field))
        elif np.issubdtype(type(field), np.float):
            new_list.append(float(field))
        else:
            new_list.append(field)
    return new_list



In [5]:
# (NO LONGER USED) created this function during experiment.
def json_paths_to_list_of_dicts(json_file_paths: List[str]) -> List[dict]:
    """
    Convert a list of JSON file paths into a list of Python dict objects.
    
    Args:
        json_file_paths: a list of JSON file paths.
        
    Returns:
        A list of dict objects
        
    Raises:
        pass
        
    References:
        Opening JSON file: https://www.geeksforgeeks.org/convert-json-to-dictionary-in-python/
        Type hinting: https://stackoverflow.com/questions/24853923/type-hinting-a-collection-of-a-specified-type
    """
    list_of_dicts = []
    for item in json_file_paths:
        with open(item) as json_file:
            data = json.load(json_file)
            list_of_dicts.append(data)
    return list_of_dicts

# song_files = get_files("/home/workspace/data/song_data")
# song_dicts = json_paths_to_list_of_dicts(song_files[:1])
# df = pd.DataFrame(song_dicts)
# df.head()

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
song_files = get_files("/home/workspace/data/song_data")
print(f"Total song files: {len(song_files)}\n")
print(f"Sample song_files:\n {song_files[:5]}\n")

Total song files: 78

Sample song_files:
 ['/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json', '/home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json', '/home/workspace/data/song_data/A/A/A/TRAAARJ128F9320760.json', '/home/workspace/data/song_data/A/A/A/TRAAAPK128E0786D96.json', '/home/workspace/data/song_data/A/A/A/TRAAABD128F429CF47.json']



In [7]:
# Read one song_file to see what we have got
song_data_df = pd.read_json(song_files[0], lines=True)
song_data_df

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


In [8]:
# For robustness let's manually set the dtype for each column
song_data_meta = {
    "dtype": {
        "artist_id": "object",
        "artist_latitude": "float64",
        "artist_location": "object",
        "artist_longitude": "float64",
        "artist_name": "object",
        "duration": "float64",
        "num_songs": "int",        
        "song_id": "object",
        "title": "object",
        "year": "int"
    } 
}

# Read one song_file to see what we have got
song_data_df = pd.read_json(song_files[0], lines=True, dtype=song_data_meta["dtype"])

# if upstream JSON is null and parsed into Pandas dataframe fields that are `np.integer` or `np.float` types.)
# .... thes null would get parsed into DataFrame as NaN (which psycopg2 would have problem parsing to psql tables)
# (note: not a problem for `object` fields) 
# ... so, instead of NaN for these numeric fields we want to replace these with Python `None`.
# psycopg2 understands None and sees it as null equivalent. It does not understand NaN however.
# This step is required following errors encountered from seeing null values for artist_latitude, artist_longitude, and year
# Ref: https://stackoverflow.com/questions/14162723/replacing-pandas-or-numpy-nan-with-a-none-to-use-with-mysqldb
# if there is a NaN in the float64 column, then that column will be changed to object to enable None value store
song_data_df = song_data_df.where(pd.notnull(song_data_df), None)
song_data_df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,None,California - LA,None,Casual,218.932,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


In [9]:
song_data_df.dtypes

artist_id           object
artist_latitude     object
artist_location     object
artist_longitude    object
artist_name         object
duration            object
num_songs            int64
song_id             object
title               object
year                 int64
dtype: object

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [10]:
song_cols = ['song_id', 'title', 'artist_id', 'year', 'duration']
song_data = list((song_data_df.loc[:, song_cols].values)[0])
print(song_data)
print([type(var) for var in song_data])

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]
[<class 'str'>, <class 'str'>, <class 'str'>, <class 'int'>, <class 'float'>]


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [11]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [12]:
artist_cols = ['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']
artist_data = list((song_data_df.loc[:, artist_cols].values)[0])
print(artist_data)
print([type(var) for var in artist_data])

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None]
[<class 'str'>, <class 'str'>, <class 'str'>, <class 'NoneType'>, <class 'NoneType'>]


Usernote: notice from the console output above... artist_data consists of just Python Native datatypes (str, None, etc.) ... these are safe to parse into psql tables.

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [13]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [14]:
log_files = get_files("/home/workspace/data/log_data")
print(f"Total log files: {len(log_files)}\n")
print(f"Sample log_files:\n {log_files[:5]}\n")

Total log files: 31

Sample log_files:
 ['/home/workspace/data/log_data/2018/11/2018-11-30-events.json', '/home/workspace/data/log_data/2018/11/2018-11-11-events.json', '/home/workspace/data/log_data/2018/11/2018-11-23-events.json', '/home/workspace/data/log_data/2018/11/2018-11-24-events.json', '/home/workspace/data/log_data/2018/11/2018-11-25-events.json']



In [15]:
# Read one log_file to see what we have got
log_data_df = pd.read_json(log_files[0], lines=True)
log_data_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [16]:
log_data_df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [17]:
log_data_df.dtypes

artist            object
auth              object
firstName         object
gender            object
itemInSession      int64
lastName          object
length           float64
level             object
location          object
method            object
page              object
registration     float64
sessionId          int64
song              object
status             int64
ts                 int64
userAgent         object
userId            object
dtype: object

In [18]:
# For robustness let's manually set the dtype for each column
# user_id (userId) is bigint in psql. max number: 9,223,372,036,854,775,807
# python3 int has no maximum value: https://note.nkmk.me/en/python-int-max-value/

log_data_meta = {
    "dtype": {
        "artist": "object",
        "auth": "object",
        "firstName": "object",
        "gender": "object",
        "itemInSession": "int64",
        "lastName": "object",
        "length": "float64",
        "level": "object",        
        "location": "object",
        "method": "object",
        "page": "object",
        "registration": "object",
        "sessionId": "int64",
        "song": "object",
        "status": "int64",
        "ts": "int64",
        "userAgent": "object",
        "userId": "bigint"
    } 
}

log_data_df = pd.read_json(log_files[0], lines=True, dtype=log_data_meta["dtype"])

# ensure NaN (not safe to parse to psql table) are converted to None (safe to parse to psql table)
log_data_df = log_data_df.where(pd.notnull(log_data_df), None)
log_data_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.857,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.54099e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.562,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.54056e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.54056e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.235,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.54056e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.764,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.54056e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [19]:
log_data_df.dtypes

artist           object
auth             object
firstName        object
gender           object
itemInSession     int64
lastName         object
length           object
level            object
location         object
method           object
page             object
registration     object
sessionId         int64
song             object
status            int64
ts                int64
userAgent        object
userId           object
dtype: object

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [20]:
next_song_df = log_data_df.loc[log_data_df["page"] == "NextSong"]
next_song_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.857,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.54099e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.562,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.54056e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.54056e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.235,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.54056e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.764,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.54056e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [21]:
# Convert current timestamp (in milliseconds) into a Pandas datatime object
# https://stackoverflow.com/questions/34883101/pandas-converting-row-with-unix-timestamp-in-milliseconds-to-datetime

t = pd.to_datetime(next_song_df['ts'], unit='ms')
t.head()

0   2018-11-30 00:22:07.796
1   2018-11-30 01:08:41.796
2   2018-11-30 01:12:48.796
3   2018-11-30 01:17:05.796
4   2018-11-30 01:20:56.796
Name: ts, dtype: datetime64[ns]

In [22]:
# To parse into postgres timestamp column, we must parse stringe format datetime in
# this strftime format: '%Y-%m-%d %H:%M:%S'
# Ref 1: https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.strftime.html'
# Ref 2: https://stackoverflow.com/questions/66197337/dataframe-with-datetime64-dtype-insert-into-to-postgressql-timestamp-column

time_data = (t.dt.strftime('%Y-%m-%d %H:%M:%S'), t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday)
column_labels = ("timestamp", "hour", "day", "week", "month", "year", "weekday")

In [23]:
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
time_df.head()

,timestamp,hour,day,week,month,year,weekday
0,2018-11-30 00:22:07,0,30,48,11,2018,4
1,2018-11-30 01:08:41,1,30,48,11,2018,4
2,2018-11-30 01:12:48,1,30,48,11,2018,4
3,2018-11-30 01:17:05,1,30,48,11,2018,4
4,2018-11-30 01:20:56,1,30,48,11,2018,4


In [24]:
# Note that the iterrows method automatically convert the np.int and np.float into Python
# native int and float types. To parse records into psql tables only native python types
# will parse. numpy numeric types are not handled by pyscopg2
for i, row in time_df.head(1).iterrows():
    print(list(row))
    print([type(var) for var in row])

['2018-11-30 00:22:07', 0, 30, 48, 11, 2018, 4]
[<class 'str'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>]


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [25]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [26]:
next_song_df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [27]:
user_cols = ['userId', 'firstName', 'lastName', 'gender', 'level']
user_df = next_song_df[user_cols]
user_df.head()

,userId,firstName,lastName,gender,level
0,91,Jayden,Bell,M,free
1,73,Jacob,Klein,M,paid
2,73,Jacob,Klein,M,paid
3,73,Jacob,Klein,M,paid
4,73,Jacob,Klein,M,paid


In [28]:
# Note that the iterrows method automatically convert the np.int and np.float into Python
# native int and float types. To parse records into psql tables only native python types
# will parse. numpy numeric types are not handled by pyscopg2
for i, row in user_df.head(1).iterrows():
    print(list(row))
    print([type(var) for var in row])

['91', 'Jayden', 'Bell', 'M', 'free']
[<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>]


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [29]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [30]:
# log dataset columns (pre-filtered on just the rows relating to the "NextSong" page)
next_song_df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [31]:
# song dataset columns (
song_data_df.columns

Index(['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year'],
      dtype='object')

##### Mini SQL Experiments - parse parameters via dict

This session is purely to get familiar with the PSQL / pscopg2 syntax

In [32]:
# do some tests with a slice from the raw song data
_song_id = song_data_df['song_id'][0]
_title = song_data_df['title'][0]
_artist_id = song_data_df['artist_id'][0]
_artist_name = song_data_df['artist_name'][0]

print(_song_id)
print(_title)
print(_artist_id)
print(_artist_name)

SOMZWCG12A8C13C480
I Didn't Mean To
ARD7TVE1187B99BFB1
Casual


In [33]:
# Experiment 1
_song_select = """SELECT songs.title FROM songs LIMIT 1;"""
cur.execute(_song_select)
results = cur.fetchone()
print(results)

("I Didn't Mean To",)


In [34]:
# Experiment 2
_song_select = """SELECT songs.song_id, songs.title FROM songs WHERE songs.song_id = %(song_id)s;"""
cur.execute(_song_select, {'song_id': _song_id})
results = cur.fetchone()
print(results)

('SOMZWCG12A8C13C480', "I Didn't Mean To")


In [35]:
# Experiment 3
_song_select = """SELECT songs.song_id, songs.title, songs.artist_id, artists.artist_id FROM songs LEFT JOIN artists ON songs.artist_id = artists.artist_id WHERE songs.title = %(song_title)s;"""
cur.execute(_song_select, {'song_title': _title})
results = cur.fetchone()
print(results)

('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 'ARD7TVE1187B99BFB1')


In [36]:
# do some tests with a slice from the raw log data (next_song_df)
_log_song_title = next_song_df['song'][0]
_log_artist_name = next_song_df['artist'][0]
_log_song_length = next_song_df['length'][0]

print(_log_song_title)
print(_log_artist_name)
print(_log_song_length)

Jim Henson's Dead
Stephen Lynch
182.85669


In [37]:
# Experiment 4
_song_select = """SELECT songs.song_id, songs.title, songs.artist_id, artists.artist_id FROM songs LEFT JOIN artists ON songs.artist_id = artists.artist_id WHERE songs.title = %(song_title)s AND artists.name = %(artist_name)s AND songs.duration = %(song_duration_s)s;"""
cur.execute(_song_select, {'song_title': _log_song_title, 'artist_name': _log_artist_name, 'song_duration_s': _log_song_length})
results = cur.fetchone()
print(results)

None


In [38]:
# Experiment 5
_song_select = """SELECT songs.song_id, songs.artist_id FROM songs LEFT JOIN artists ON songs.artist_id = artists.artist_id WHERE songs.title = %(song_title)s AND artists.name = %(artist_name)s AND songs.duration = %(song_duration_s)s;"""
cur.execute(_song_select, {'song_title': _log_song_title, 'artist_name': _log_artist_name, 'song_duration_s': _log_song_length})
results = cur.fetchone()
print(results)

None


In [39]:
# Experiment 6
_song_select = """SELECT songs.song_id, songs.artist_id FROM songs INNER JOIN artists ON songs.artist_id = artists.artist_id WHERE songs.title = %(song_title)s AND artists.name = %(artist_name)s AND songs.duration = %(song_duration_s)s;"""
cur.execute(_song_select, {'song_title': _log_song_title, 'artist_name': _log_artist_name, 'song_duration_s': _log_song_length})
results = cur.fetchone()
print(results)

None


In [40]:
# Experiment 7
_song_select = """\
SELECT songs.song_id, songs.artist_id \
FROM songs \
INNER JOIN artists 
ON songs.artist_id = artists.artist_id \
WHERE \
songs.title = %(song_title)s AND \
artists.name = %(artist_name)s AND \
songs.duration = %(song_duration_s)s;\
"""
for index, row in next_song_df.head().iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(_song_select, {
        'song_title': row.song,
        'artist_name': row.artist,
        'song_duration_s': row.length
    })
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.page, time_data[0][index], row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    print(songplay_data)

('NextSong', '2018-11-30 00:22:07', '91', 'free', None, None, 829, 'Dallas-Fort Worth-Arlington, TX', 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0)')
('NextSong', '2018-11-30 01:08:41', '73', 'paid', None, None, 1049, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
('NextSong', '2018-11-30 01:12:48', '73', 'paid', None, None, 1049, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
('NextSong', '2018-11-30 01:17:05', '73', 'paid', None, None, 1049, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
('NextSong', '2018-11-30 01:20:56', '73', 'paid', None, None, 1049, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; 

In [41]:
# Experiment 7 - parset parameter via tuple
_song_select = """SELECT songs.song_id, songs.artist_id FROM songs INNER JOIN artists ON songs.artist_id = artists.artist_id WHERE songs.title = %s AND artists.name = %s AND songs.duration = %s;"""
cur.execute(_song_select, (_log_song_title, _log_artist_name, _log_song_length))
results = cur.fetchone()
print(results)

None


##### More SQL Experiments - parse parameters via tuple

In [42]:
# parse parameter via tuple
_song_select = """\
SELECT songs.song_id, songs.artist_id \
FROM songs \
INNER JOIN artists \
ON songs.artist_id = artists.artist_id \
WHERE songs.title = %s AND \
artists.name = %s AND \
songs.duration = %s;\
"""

for index, row in next_song_df.head().iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(_song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.page, time_data[0][index], row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    print(songplay_data)

('NextSong', '2018-11-30 00:22:07', '91', 'free', None, None, 829, 'Dallas-Fort Worth-Arlington, TX', 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0)')
('NextSong', '2018-11-30 01:08:41', '73', 'paid', None, None, 1049, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
('NextSong', '2018-11-30 01:12:48', '73', 'paid', None, None, 1049, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
('NextSong', '2018-11-30 01:17:05', '73', 'paid', None, None, 1049, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
('NextSong', '2018-11-30 01:20:56', '73', 'paid', None, None, 1049, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; 

#### Populate `songplay` psql table for real

For consistency within `sql_queries.py` we will stick with tuple parameter style stytax.

In [43]:
# this is our song_select query from sql_queries.py
song_select

'SELECT songs.song_id, songs.artist_id FROM songs INNER JOIN artists ON songs.artist_id = artists.artist_id WHERE songs.title = %s AND artists.name = %s AND songs.duration = %s;'

In [44]:
# parse parameter via tuple
# below is the Udacity template code.
for index, row in next_song_df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.page, time_data[0][index], row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
#     print(songplay_data)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [45]:
cur.close()
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.